In [ ]:
def black_scholes_call(S0, K, T, r, sigma):
    """
    Calculate the price of a European call option using the Black-Scholes formula.
    
    Parameters:
    S0 (float): Current stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (annualized)
    sigma (float): Volatility of the underlying asset (annualized)
    
    Returns:
    float: Call option price
    """
    d1 = (np.log(S0/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    return call_price

# Example usage
S0 = 100    # Current stock price
K = 100     # Strike price
T = 1       # Time to maturity (in years)
r = 0.05    # Risk-free rate
sigma = 0.2 # Volatility

call_price = black_scholes_call(S0, K, T, r, sigma)
print(f"European Call Option Price: ${call_price:.2f}")

def gbm_multiple_paths(S0, mu, sigma, T, N, M, random_seed=None):
    """
    Simulate multiple paths of Geometric Brownian Motion.
    
    Parameters:
    S0 (float): Initial stock price
    mu (float): Drift coefficient (annualized)
    sigma (float): Volatility (annualized)
    T (float): Time horizon in years
    N (int): Number of time steps
    M (int): Number of paths to simulate
    random_seed (int): Seed for random number generator
    
    Returns:
    tuple: Time points and matrix of simulated stock prices (each column is a path)
    """
    if random_seed is not None:
        np.random.seed(random_seed)
        
    dt = T/N
    t = np.linspace(0, T, N+1)
    
    # Initialize price matrix (rows=time steps, columns=paths)
    S = np.zeros((N+1, M))
    S[0] = S0
    
    # Generate random normal values for all paths at once
    Z = np.random.normal(0, 1, size=(N, M))
    
    # Simulate all paths
    for i in range(1, N+1):
        S[i] = S[i-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[i-1])
    
    return t, S

def plot_gbm_dist_multid(S0: float, mu: float, sigma: float, T: float) -> None:
    """
    Plots the theoretical probability density function (PDF) of Geometric Brownian Motion (GBM)
    as an interactive 3D surface over price and time using Plotly.
    """
    # Define ranges for stock price (S) and time (t).
    t_values: np.ndarray = np.linspace(1e-9, T, 100)

    # Define a reasonable upper bound for the stock price axis.
    S_upper_bound: float = S0 * np.exp((mu + 3 * sigma) * T)
    if S_upper_bound < S0 * 2:
        S_upper_bound = S0 * 2
    S_values: np.ndarray = np.linspace(1e-9, S_upper_bound, 100)

    # Create a meshgrid.
    S_grid, t_grid = np.meshgrid(S_values, t_values)

    # Calculate lognormal PDF parameters.
    s_param: np.ndarray = sigma * np.sqrt(t_grid)
    scale_param: np.ndarray = S0 * np.exp((mu - 0.5 * sigma**2) * t_grid)

    # Calculate PDF values, handling t near 0.
    pdf_values: np.ndarray = np.zeros_like(S_grid)
    positive_t_mask: np.ndarray = t_grid > 1e-9

    pdf_values[positive_t_mask] = lognorm.pdf(
        S_grid[positive_t_mask],
        s=s_param[positive_t_mask],
        scale=scale_param[positive_t_mask]
    )

    # Create Plotly figure.
    fig = go.Figure(data=[go.Surface(z=pdf_values, x=S_grid, y=t_grid)])

    fig.update_layout(
        title=f'GBM Probability Distribution (S0={S0}, μ={mu}, σ={sigma})',
        scene = dict(
            xaxis_title=r'Stock Price ($S_t$)',
            yaxis_title=r'Time ($t$)',
            zaxis_title=r'Probability Density $P(S_t, t)$'
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    # Show the plot.
    fig.show()

plot_gbm_dist_multid(S0=100, mu=0.1, sigma=0.2, T=1)

def monte_carlo_call(S0, K, T, r, sigma, M=10000):
    """
    Price a European call option using Monte Carlo simulation with GBM.
    
    Parameters:
    S0 (float): Current stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (annualized)
    sigma (float): Volatility (annualized)
    M (int): Number of simulations
    
    Returns:
    float: Estimated call option price
    """
    
    _, S = gbm_multiple_paths(S0, mu=r, sigma=sigma, T=T, N=1, M=M)
    
    # Calculate payoffs at maturity
    payoffs = np.maximum(S[-1] - K, 0)
    
    # Discount payoffs to present value
    option_price = np.exp(-r * T) * np.mean(payoffs)
    
    return option_price

# Compare Black-Scholes and Monte Carlo prices
mc_price = monte_carlo_call(S0, K, T, r, sigma)
bs_price = black_scholes_call(S0, K, T, r, sigma)

print(f"Black-Scholes Call Price: ${bs_price:.4f}")
print(f"Monte Carlo Call Price:   ${mc_price:.4f}")
print(f"Difference:               ${abs(bs_price - mc_price):.4f}")

In [ ]:
def iv_surface(S0: float, r: float, sigma_base: float, plots_dir: str) -> None:
    """
    Plots a stylized implied volatility surface as an interactive 3D plot using Plotly.

    The surface shows implied volatility as a function of strike price and time to expiration.
    A stylized smile and term structure are generated based on the base volatility.

    Args:
        S0 (float): The current stock price.
        r (float): The risk-free interest rate.
        sigma_base (float): A base volatility around which the surface is built (e.g., ATM volatility for a medium maturity).
        plots_dir (str): The directory to save the HTML plot file.
    """
    # Define ranges for strike price (K) and time to expiration (T).
    # Strike prices relative to S0 (moneyness).
    moneyness_values: np.ndarray = np.linspace(0.7, 1.3, 50) # Range from 70% to 130% of S0
    K_values: np.ndarray = S0 * moneyness_values

    # Times to expiration in years.
    T_values: np.ndarray = np.linspace(0.1, 2.0, 50) # Range from 0.1 to 2.0 years

    # Create a meshgrid for K and T.
    K_grid, T_grid = np.meshgrid(K_values, T_values)

    # Define parameters for the stylized smile and term structure.
    # These parameters determine the shape of the surface.
    smile_curvature: float = 0.8 # Controls how pronounced the smile is (higher value means more curvature)
    term_structure_slope: float = 0.1 # Controls how volatility changes with maturity (positive for upward slope)

    # Calculate stylized implied volatility for each (K, T) pair.
    # Using a simple model: base_vol * (1 + smile_term + term_structure_term)
    # Smile term based on squared deviation from ATM (K=S0).
    smile_term: np.ndarray = smile_curvature * ((K_grid / S0) - 1)**2
    # Term structure term based on time to expiration.
    term_structure_term: np.ndarray = term_structure_slope * T_grid

    # Calculate implied volatility grid, ensuring it's non-negative.
    implied_vol_grid: np.ndarray = sigma_base * (1 + smile_term + term_structure_term)
    implied_vol_grid[implied_vol_grid < 1e-9] = 1e-9 # Ensure non-negative volatility

    # Create Plotly figure with a 3D surface trace.
    fig = go.Figure(data=[go.Surface(z=implied_vol_grid, x=K_grid, y=T_grid)])

    # Update layout for a professional look and set axis labels and title.
    fig.update_layout(
        title='Stylized Implied Volatility Surface',
        scene = dict(
            xaxis_title='Strike Price (K)',
            yaxis_title='Time to Expiration (T)',
            zaxis_title='Implied Volatility (σ_implied)'
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    # Ensure plots directory exists
    if not os.path.exists(plots_dir):
        os.makedirs(plots_dir)

    # Define file path for saving
    plot_filepath: str = os.path.join(plots_dir, 'stylized_iv_surface.html')

    # Save the figure to an HTML file
    fig.write_html(plot_filepath)

    print(f"Stylized IV Surface plot saved to {plot_filepath}")

iv_surface(S0=100, r=0.05, sigma_base=0.2, plots_dir=plots_dir)

def vol_smile(S0: float, r: float, sigma_base: float, T_fixed: float, plots_dir: str) -> None:
    """
    Plots a stylized implied volatility smile for a fixed time to expiration using Plotly.

    The smile shows implied volatility as a function of strike price for a given maturity.
    A stylized smile shape is generated based on the base volatility.

    Args:
        S0 (float): The current stock price.
        r (float): The risk-free interest rate.
        sigma_base (float): A base volatility around which the smile is built (e.g., ATM volatility).
        T_fixed (float): The fixed time to expiration (in years) for the smile.
        plots_dir (str): The directory to save the HTML plot file.
    """
    # Define range for strike price (K) relative to S0 (moneyness).
    moneyness_values: np.ndarray = np.linspace(0.7, 1.3, 100) # Range from 70% to 130% of S0
    K_values: np.ndarray = S0 * moneyness_values

    # Define parameters for the stylized smile at the fixed maturity.
    smile_curvature: float = 0.5 # Controls how pronounced the smile is (higher value means more curvature)

    # Calculate stylized implied volatility for each K at the fixed T.
    # Using a simple model: base_vol * (1 + smile_term) at fixed T.
    # Smile term based on squared deviation from ATM (K=S0).
    smile_term: np.ndarray = smile_curvature * ((K_values / S0) - 1)**2

    # Calculate implied volatility values, ensuring they are non-negative.
    implied_vol_values: np.ndarray = sigma_base * (1 + smile_term)
    implied_vol_values[implied_vol_values < 1e-9] = 1e-9 # Ensure non-negative volatility


    # Create Plotly figure with a 2D scatter trace representing the smile.
    fig = go.Figure(data=[go.Scatter(x=K_values, y=implied_vol_values, mode='lines')])

    # Update layout for a professional look and set axis labels and title.
    fig.update_layout(
        title=f'Stylized Implied Volatility Smile at T = {T_fixed:.2f} years',
        xaxis_title='Strike Price (K)',
        yaxis_title='Implied Volatility (σ_implied)',
        hovermode='x unified' # Add hover effect for better interactivity
    )

    # Ensure plots directory exists
    if not os.path.exists(plots_dir):
        os.makedirs(plots_dir)

    # Define file path for saving
    plot_filepath: str = os.path.join(plots_dir, f'stylized_vol_smile_T{T_fixed:.2f}.html')

    # Save the figure to an HTML file
    fig.write_html(plot_filepath)

    print(f"Stylized Volatility Smile plot saved to {plot_filepath}")

vol_smile(S0=100, r=0.05, sigma_base=0.2, T_fixed=1.0, plots_dir=plots_dir)